In [1]:
import numpy as np 

In [4]:
def load_dataset():
    data_mat = []; label_mat = []
    with open('testSet.txt') as fr:
        for line in fr.readlines():
            line_arr = line.strip().split()
            data_mat.append([1.0, float(line_arr[0]), float(line_arr[1])])
            label_mat.append(int(line_arr[2]))
    return data_mat, label_mat

def sigmoid(inx):
    return 1.0/(1 + np.exp(-inx))

def grad_ascent(data_mat_in, classlabels):
    data_mat = np.mat(data_mat_in)
    label_mat = np.mat(classlabels).transpose()
    m, n = np.shape(data_mat)
    alpha = 0.001
    max_cycles = 500
    weights = np.ones((n,1))
    for k in range(max_cycles):
        h = sigmoid(data_mat * weights)
        error = (label_mat - h)
        weights = weights + alpha * data_mat.transpose() * error 
    return weights

In [5]:
data_arr, label_mat = load_dataset()
grad_ascent(data_arr, label_mat)

matrix([[ 4.12414349],
        [ 0.48007329],
        [-0.6168482 ]])